In [1]:
import logging
import logging.handlers
import os
import io
import pandas as pd
import numpy as np
import requests
import json
# from chessdotcom import get_player_profile, get_player_stats, get_player_game_archives  (not working)
import chess.pgn
from converter.pgn_data import PGNData
import time
from datetime import date
from datetime import datetime
from dateutil.relativedelta import relativedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g
import sys
#import chess_dot_com_api as capi
import requests
from datetime import date
from datetime import datetime
import json
from dateutil.relativedelta import relativedelta
#import googlesheet as g
#import data
#import ratings
from pgn_parser import pgn, parser

In [2]:
students_username = ['yaohengli',
           'chengliam',
           'blakey0313',
           'akfunchess66',

           'Leo20166',
           'Jasminezhao777',
           'Justinzhao777',
           'Spade',
           'Jason-Ma',
           'Dingcheng0',
           
           'chessloverma',
           'AndyAAYY',          
           'whatwhywhywhat',
           'charliezienyang',
           'TheChessBoi9999',
           'evejyl77',
           'Gavin_ML']

In [3]:
student_df = pd.read_csv("../../Desktop/rcc_summer_2024/RCC_2024_summer_student.csv")

In [4]:
student_df.head()

,class,username,real_name
0,Cowboy,oayz01,Aiden Zhang
1,Cowboy,IowaPawn2018,Shilang Zhu
2,Cowboy,Claire_youyou,Claire Yan
3,Cowboy,RayX93,Ray Xia
4,Cowboy,MasonYD,Mason Dong


In [5]:
def last_n_month(n):
    """
    purpose:
    return the month as yyyy/mm format of the past n months from now
    
    input -
    n: number of months from past
    
    output -
    a list of month with yyyy/mm format
    """
    months_lst = []
    for num in range(n):
        months = date.today() + relativedelta(months=-num)
        if months.month <= 9:
            months_lst.append(str(months.year) + "/"+ "0" + str(months.month))
        else:
            months_lst.append(str(months.year) + "/"+ str(months.month))
    return months_lst


def get_archive_games(filename,
                     user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}):
    """
    purpose:
    
    return games in one archive file
    
    input:
    filename - filename that contains game urls
    
    output: 
    """
    games = requests.get(filename,headers = user_agent).json()['games']
    return games

def lowercase_student(student_list):
    """
    to lowercase all the username
    
    input - list, list of student username, regardless upper or lower case
    
    output - list, list of student username, lower case
    """
    
    lower_students = [x.lower() for x in student_list]
    
    return lower_students

In [6]:
def get_user_archives(username, 
                      start_date = '2024-01-01',
                      end_date = '2024-07-01',
                     user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}):
    """
    purpose:
    get archive monthly files of specific chess.com player
    
    input:
    username - username of the chess.com player
    nr_months - integer, nummber of past months that we want to get the archives
    # to request chess.com API
       user_agent = {'User-Agent': 'username: tianminlyu, email: tianminlyu@gmail.com'}
    
    output:
    target_month - files of archives according to months parameter
    """
    url = "https://api.chess.com/pub/player/{username}/games/archives".format(username = username)
    archive_request = requests.get(url, headers = user_agent)
    archives = archive_request.json()['archives']
    target_archive = []
    month_format = '%Y-%m-%d'
    start_date = datetime.strptime(start_date, month_format)
    end_date = datetime.strptime(end_date, month_format)

    for archive in archives:
        archive_month = str(archive[-7:][:4]) + "-" + str(archive[-7:][5:]) + "-01"
 
        # convert from string format to datetime format
        datetime_month = datetime.strptime(archive_month, month_format)
        if datetime_month >= start_date and datetime_month <= end_date:
            target_archive.append(archive)
    return target_archive

tianmin_archive = get_user_archives('tianminlyu')

In [7]:
# Latest rating of rapid game, initial setup, rated game
def latest_rating(archives,start_date,end_date):
    for archive in [x for x in archives[::-1]]:
        games = get_archive_games(archive)
        #print(archive)
        #print(len(games))
        for game in games[::-1]:
            end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
            if end_time <= end_date and end_time >= start_date and game['initial_setup'] == 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1' and game['rated'] == True and game['time_class'] == 'rapid':
                #print(end_time)
                if archive.split("/")[5].lower() == game['white']['username'].lower():
                    rating = game['white']['rating']
                else:
                    rating = game['black']['rating']
                return rating
                break
#latest_rating(chesslover, '2024-01-01', '2024-06-12')

In [8]:
# begin rating of rapid game, initial setup, rated game
def begin_rating(archives,start_date,end_date):
    for archive in [x for x in archives]:
        games = get_archive_games(archive)
        #print(archive)
        #print(len(games))
        for game in games:
            end_time = datetime.utcfromtimestamp(game['end_time']).strftime('%Y-%m-%d %H:%M:%S')
            if end_time <= end_date and end_time >= start_date and game['initial_setup'] == 'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1' and game['rated'] == True and game['time_class'] == 'rapid':
                #print(end_time)
                #print(game['white']['username'])
                #print(game['black']['username'])
                if archive.split("/")[5].lower() == game['white']['username'].lower():
                    rating = game['white']['rating']
                else:
                    rating = game['black']['rating']
                return rating
                break

In [ ]:
def get_student_performance(student_df, start_date= '2024-01-01', end_date= '2024-06-12'):
    class_names = []
    student_usernames = []
    user_realnames = []
    user_begin_ratings = []
    user_latest_ratings = []
    pro_start_time = time.time()
    for i in (range(student_df.shape[0])):
        print(str(i) + "/" + str(student_df.shape[0]))
        class_name = student_df.iloc[i]['class']
        student_username = student_df.iloc[i]['username'].lower()
        print(student_username)
        student_realname = student_df.iloc[i]['real_name']
        archives = get_user_archives(student_username, start_date= start_date, end_date= end_date)
        user_begin_rating = begin_rating(archives, start_date = start_date, end_date = end_date)
        user_latest_rating = latest_rating(archives, start_date = start_date, end_date = end_date)
        
        class_names.append(class_name)
        student_usernames.append(student_username)
        user_realnames.append(student_realname)
        user_begin_ratings.append(user_begin_rating)
        user_latest_ratings.append(user_latest_rating)
        
    pro_end_time = time.time()
    run_time = pro_end_time - pro_start_time
    
    return class_names, student_usernames, user_realnames, user_begin_ratings, user_latest_ratings, run_time

get_student_performance_lst = get_student_performance(student_df)
print(get_student_performance_lst[5])

0/544
oayz01
1/544
iowapawn2018
2/544
claire_youyou
3/544
rayx93
4/544
masonyd
5/544
chess_zz2023
6/544
andrew-wang-2024
7/544
cateleyn
8/544
danielrxj
9/544
gm_destroyer471_lol
10/544
chess_zz2023
11/544
oscar_zh
12/544
bobogaganoob
13/544
ml1363
14/544
enzolovechess
15/544
zhangjunyu2017
16/544
lucas628shen
17/544
chenzoe0901
18/544
sadiegege
19/544
kevin_tangkc
20/544
17cloud
21/544
junjuans1990
22/544
chloebeat
23/544
jessica-chen
24/544
albertchw
25/544
elisa-wang
26/544
raphael_fang
27/544
iriszxr
28/544
vinali1
29/544
chickykid1
30/544
lucas101516
31/544
hugoxuedl
32/544
gloriawu2023


In [ ]:
df = pd.DataFrame()
df['class_name'] = get_student_performance_lst[0]    
df['username'] = get_student_performance_lst[1]
df['real_name'] = get_student_performance_lst[2]
df['begin_rating'] = get_student_performance_lst[3]
df['latest_rating'] = get_student_performance_lst[4]
df['change'] = df['latest_rating'] - df['begin_rating']
#df['start_date'] = start_date
#df['end_date'] = end_date
df = df.drop_duplicates()


In [ ]:
df.to_csv("../../Desktop/rcc_summer_2024/RCC_2024_summer_student_performance_output.csv")

In [ ]:
df.head()